In [1]:
import tensorflow as tf, os, pandas as pd, numpy as np, pickle, glob
from sklearn.model_selection import KFold
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

2023-04-04 16:25:02.829769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 16:25:02.955606: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-04 16:25:03.476332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-04 16:25:03.476407: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
USD_data_train_pkl = glob.glob("/ssd8/chih/project/yadong/predict_baseline_version01/dataset/PeriHD_pickle_save/USD_sample2693_only-merge-sequence_fullcontext/train/*.pkl")
USD_data_test_pkl = glob.glob("/ssd8/chih/project/yadong/predict_baseline_version01/dataset/PeriHD_pickle_save/USD_sample2693_only-merge-sequence_fullcontext/test/*.pkl")

In [3]:
def pk2pf(pkl_list):
    data_train = []
    for pk in pkl_list:
        with open(pk, 'rb') as f:
            pk_dict = pickle.load(f)
        data_train.append(pk_dict)
    data_train = pd.DataFrame(data_train)
    
    return data_train
USD_data_train = pk2pf(USD_data_train_pkl)
USD_data_test = pk2pf(USD_data_test_pkl)

In [4]:
# USD_data_train_embedding = np.array(USD_data_train['full_embed'].tolist())
USD_data_test_embedding = np.array(USD_data_test['full_embed'].tolist())

/tmp/ipykernel_40726/3208931757.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  USD_data_test_embedding = np.array(USD_data_test['full_embed'].tolist())


In [5]:
# print(len(USD_data_train.label.values)-sum(USD_data_train.label.values), sum(USD_data_train.label.values))
print(len(USD_data_test.label.values)-sum(USD_data_test.label.values), sum(USD_data_test.label.values))

74 26


In [6]:
import keras
def create_model(input_shape = (1024,)):
  model = tf.keras.Sequential([
    keras.layers.Dense(256, activation='relu', input_shape = input_shape),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
  ])
  model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['acc'])
  return model

In [1]:
from sklearn.metrics import classification_report, roc_curve, precision_recall_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
import matplotlib.pylab as plt
from tqdm import tqdm
def get_weight_path(fold, train_type):
    # return './model_weight/weights_periHD_KFold={}[Fusion].h5'.format(fold)
    # return './model_weight/PeriHD/weights_periHD_KFold={}[{}-only-merge-sequnece-resplit].h5'.format(fold,train_type)
    return './model_weight/PeriHD/weights_periHD_KFold={}[{}-only-merge-sequnece-resplit-periHD-neg+pos].h5'.format(fold,train_type)
kfold = KFold(n_splits=5, shuffle=True)
gt_in = USD_data_test.label.values
gt_ex = USD_data_test.label.values
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
CM_internal = []
CM_external = []
train_type = 'USD'
input_shape  = (512,)
Fusion_intranal_embeddings = USD_data_test_embedding
Fusion_external_validation_embeddings = USD_data_test_embedding
false_pred = pd.DataFrame([], columns=['index', 'Pred score', 'Answer', 'Sentence'])
for fold in ['1','2','3','4','5']:
# for fold in ['1']:
    weight_name = get_weight_path(fold, train_type)
    model_cls = create_model(input_shape=input_shape)
    model_cls.load_weights(weight_name)
    # Internal metric
    # split_data_internal = {"pred":[], "truth": []}
    # train_pred_seq_stack =[]
    # for (seq_stack, ans) in tqdm(zip(Fusion_intranal_embeddings, gt_in)):
    #     test_pred = model_cls.predict(seq_stack,verbose=0)
    #     train_pred_seq_stack.append(test_pred)
    #     split_data_internal['pred'].append(np.max(test_pred))
    #     split_data_internal['truth'].append(ans)
    # test_binary_pred1 = (np.array(split_data_internal['pred'])>0.5).astype(np.int8)
    # tn, fp, fn, tp = confusion_matrix(split_data_internal['truth'], test_binary_pred1).ravel()
    # CM_internal.append([tn, fp, fn, tp])
    # fpr, tpr, thresholds = roc_curve(split_data_internal['truth'], split_data_internal['pred'], pos_label=1)
    # auc = roc_auc_score(split_data_internal['truth'], split_data_internal['pred'])
    # axs[0].plot(fpr, tpr, label = "Test Fold={} AUC: {}".format(fold, round(auc, 4)))
    # External metric
    split_data_external = {"pred":[], "truth": [], "index": []}
    test_pred_seq_stack =[]
    for (seq_stack, ans) in tqdm(zip(Fusion_external_validation_embeddings, gt_ex)):
        test_pred = model_cls.predict(seq_stack, verbose=0)
        test_pred_seq_stack.append(test_pred)
        split_data_external['pred'].append(np.max(test_pred))
        split_data_external['index'].append(np.argmax(test_pred))
        split_data_external['truth'].append(ans)
    test_binary_pred2 = (np.array(split_data_external['pred'])>0.5).astype(np.int8)
    tn, fp, fn, tp = confusion_matrix(split_data_external['truth'], test_binary_pred2).ravel()
    CM_external.append([tn, fp, fn, tp])
    fpr, tpr, thresholds = roc_curve(split_data_external['truth'], split_data_external['pred'], pos_label=1)
    auc = roc_auc_score(split_data_external['truth'], split_data_external['pred'])
    axs[1].plot(fpr, tpr, label = "Test Fold={} AUC: {}".format(fold, round(auc, 4)))

    for idx, (ex_ans, ex_pre, ex_index) in enumerate(zip(split_data_external['truth'], split_data_external['pred'],split_data_external['index'])):
        temp = pd.DataFrame([], columns=['index', 'Answer', 'Pred score', 'Sentence'])
        temp['index'] = [idx]
        temp['Answer'] = [ex_ans]
        temp['Pred score'] = [ex_pre]
        temp['Sentence'] = [USD_data_test.iloc[idx].full_text[ex_index]]
        false_pred = pd.concat([false_pred, temp])
        
axs[0].plot([0,1],[0,1], linestyle='--')
axs[1].plot([0,1],[0,1], linestyle='--')
axs[0].legend()
axs[1].legend()
axs[0].set_xlabel("FPR")
axs[0].set_ylabel("TPR")
axs[1].set_xlabel("FPR")
axs[1].set_ylabel("TPR")
axs[0].set_title(f"[Internal][{train_type}]PeriHD full-sentence ROC-Curve")
axs[1].set_title(f"[External][{train_type}]PeriHD full-sentence ROC-Curve")


NameError: name 'KFold' is not defined

In [18]:
false_pred_ths_result = (false_pred['Pred score']>0.5).astype(int)
false_pred['Pred Result'] = false_pred_ths_result
false_pred_drop = false_pred.drop_duplicates(subset="Sentence")

In [26]:
false_pred_drop.to_csv('./外部資料Sample預測錯誤結果_Negative和Positive皆做merge.csv', encoding='utf-8-sig', index=False)

In [11]:
# 已標記資料效能評估
def metric_calc(array_):
    tn, fp, fn, tp = array_
    acc = (tp+tn)/(tp+tn+fn+fp)
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    prec = tp/(tp+fp)
    f1 = (2*prec*sens)/(prec+sens)
    return acc, prec, sens, spec, f1

def get_metric(arr):
    mean_arr = np.mean(np.array(arr), axis=0)
    std_arr = np.std(np.array(arr), axis=0)
    return mean_arr, std_arr

In [12]:

# CM_train = np.array(CM_internal)
CM_train = np.array(CM_external)
CM_test = np.array(CM_external)
T_stack, t_stack = [],[]
for T_ in CM_train:
    T_stack.append(metric_calc(T_))
    
for t_ in CM_test:
    t_stack.append(metric_calc(t_))
#內部資料 - Train & Test
mark_train_mean, mark_train_std = get_metric(T_stack)
mark_test_mean, mark_test_std = get_metric(t_stack)
mark_train = pd.DataFrame(['Accuracy','Precision', 'Sensitivity', 'Specificity', 'F1-score'], columns=[f'{train_type} Metric Train'])
for T_, f in zip(T_stack,['fold-1','fold-2','fold-3','fold-4','fold-5']):mark_train[f] = np.round(T_,4)
mark_train['Mean'] = np.round(mark_train_mean,4)
mark_train['Std'] = np.round(mark_train_std,4)

mark_test = pd.DataFrame(['Accuracy','Precision', 'Sensitivity', 'Specificity', 'F1-score'], columns=[f'{train_type} Metric Test'])
for t_, f in zip(t_stack,['fold-1','fold-2','fold-3','fold-4','fold-5']):mark_test[f] = np.round(t_,4)
mark_test['Mean'] = np.round(mark_test_mean,4)
mark_test['Std'] = np.round(mark_test_std,4)

In [13]:
mark_train

,USD Metric Train,fold-1,fold-2,fold-3,fold-4,fold-5,Mean,Std
0,Accuracy,0.9400,0.9400,0.9500,0.9400,0.9300,0.9400,0.0063
1,Precision,0.8125,0.8125,0.8387,0.8125,0.7879,0.8128,0.0161
2,Sensitivity,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
3,Specificity,0.9189,0.9189,0.9324,0.9189,0.9054,0.9189,0.0085
4,F1-score,0.8966,0.8966,0.9123,0.8966,0.8814,0.8967,0.0098


In [14]:
mark_test

,USD Metric Test,fold-1,fold-2,fold-3,fold-4,fold-5,Mean,Std
0,Accuracy,0.9400,0.9400,0.9500,0.9400,0.9300,0.9400,0.0063
1,Precision,0.8125,0.8125,0.8387,0.8125,0.7879,0.8128,0.0161
2,Sensitivity,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
3,Specificity,0.9189,0.9189,0.9324,0.9189,0.9054,0.9189,0.0085
4,F1-score,0.8966,0.8966,0.9123,0.8966,0.8814,0.8967,0.0098
